# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix (q=0,ω=0) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234990095818                   -0.50    8.0
  2   -7.250351942254       -1.81       -1.40    1.0
  3   -7.251247508493       -3.05       -2.04    3.0
  4   -7.251079227708   +   -3.77       -1.96    4.0
  5   -7.251332459046       -3.60       -2.74    3.0
  6   -7.251338103472       -5.25       -3.14    3.0
  7   -7.251338785698       -6.17       -4.09    2.0
  8   -7.251338793474       -8.11       -4.20    4.0
  9   -7.251338798536       -8.30       -4.80    3.0
 10   -7.251338798619      -10.08       -5.03    1.0
 11   -7.251338798702      -10.08       -5.84    3.0
 12   -7.251338798704      -11.73       -6.16    3.0
 13   -7.251338798705      -12.42       -6.80    2.0
 14   -7.251338798705      -13.59       -7.17    4.0
 15   -7.251338798705      -14.45       -7.47    2.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [1, 1, 1]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Apply ε† = 1 - χ0 (vc + fxc)

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end

eps_fun (generic function with 1 method)

eager diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.05158134849190125
[ Info: Arnoldi iteration step 2: normres = 0.6184373610627877
[ Info: Arnoldi iteration step 3: normres = 0.5650073463791565
[ Info: Arnoldi iteration step 4: normres = 0.3522239213777326
[ Info: Arnoldi iteration step 5: normres = 0.30977205315729905
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (9.33e-03, 7.91e-02, 2.22e-01, 2.00e-01, 1.67e-02)
[ Info: Arnoldi iteration step 6: normres = 0.5122040611758517
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (3.70e-03, 1.22e-01, 4.65e-01, 1.22e-01, 8.47e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07914684327664986
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.66e-04, 2.09e-02, 2.94e-02, 3.49e-02, 5.43e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11380460060653652
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (8.11e-